<a href="https://colab.research.google.com/github/LasTAD/pysyft_experiments/blob/main/PySyft_Exp_DO_small_signal_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подготовка среды к выполнению.

In [1]:

# This only runs in colab and clones the code sets it up and fixes a few issues, 
# you can skip this if you are running Jupyter Notebooks
import sys
if "google.colab" in sys.modules:
    branch = "dev"    # change to the branch you want
    ! git clone --single-branch --branch $branch https://github.com/OpenMined/PySyft.git
    ! cd PySyft && ./scripts/colab.sh      # fixes some colab python issues
    sys.path.append("/content/PySyft/src") # prevents needing restart

Cloning into 'PySyft'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 55872 (delta 34), reused 80 (delta 32), pack-reused 55761
Receiving objects: 100% (55872/55872), 58.87 MiB | 21.38 MiB/s, done.
Resolving deltas: 100% (37557/37557), done.
> git pull
> using latest commit
  83f0fa85 Merge pull request #5297 from madhavajay/madhava/fix_readme
> installing pip requirements
> patching python 3.6.9 colab bug
sed: can't read /usr/local/lib/python3.6/dist-packages/typeguard/__init__.py: No such file or directory
> colab environment ready


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/My Drive/Диплом Кобызев Н.П. 5374/PySyft/Эксперименты/Dataset

/content/drive/My Drive/Диплом Кобызев Н.П. 5374/PySyft/Эксперименты/Dataset


In [ ]:
ls

opel_corsa_01.csv  opel_corsa_02.csv  peugeot_207_01.csv  peugeot_207_02.csv


Подготовка датасета

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import torch.utils.data

In [ ]:
df1=pd.read_csv('opel_corsa_01.csv', sep=';',decimal=',')
df2=pd.read_csv('opel_corsa_02.csv', sep=';',decimal=',')
df3=pd.read_csv('peugeot_207_01.csv', sep=';',decimal=',')
df4=pd.read_csv('peugeot_207_02.csv', sep=';',decimal=',')
df_Dataset=pd.concat([df1,df2,df3,df4],axis=0)
df_Dataset.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            4613, 4614, 4615, 4616, 4617, 4618, 4619, 4620, 4621, 4622],
           dtype='int64', length=24957)

In [ ]:
if (df_Dataset.isnull().sum().sum()!=0):
    df_Dataset.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
df_Dataset.index

Int64Index([  59,   60,   61,   62,   63,   64,   65,   66,   67,   68,
            ...
            4613, 4614, 4615, 4616, 4617, 4618, 4619, 4620, 4621, 4622],
           dtype='int64', length=23762)

In [ ]:
y = df_Dataset['drivingStyle']
X = df_Dataset.drop(['drivingStyle','traffic','roadSurface'], axis=1)

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0)

y.shape
X.shape

(23762, 14)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

feature_scaler = MinMaxScaler()
train_X = feature_scaler.fit_transform(train_X)
test_X = feature_scaler.transform(test_X)

In [ ]:
Xtrain, Xval, Ytrain, Yval = train_test_split(train_X, train_y, test_size=0.2, random_state=5)

In [ ]:
print(train_X)

[[0.67422128 0.18478262 0.17717839 ... 0.01754386 0.15135287 0.29033371]
 [0.68271951 0.71681787 0.47275737 ... 0.47368421 0.10151681 0.11742405]
 [0.68555168 0.47463769 0.4167775  ... 0.21052632 0.11985281 0.17308341]
 ...
 [0.61756364 0.4112319  0.46320641 ... 0.15789474 0.11720648 0.02055666]
 [0.61756364 0.35507245 0.26069473 ... 0.28070175 0.14888521 0.26056512]
 [0.70254988 0.13405797 0.18209657 ... 0.24561404 0.13815017 0.10133905]]


In [ ]:
train_X.shape

(19009, 14)

In [ ]:
train_y[train_y=='AggressiveStyle']=0
train_y[train_y=='EvenPaceStyle']=1
train_y


7127    0
6664    1
117     1
5834    1
4302    1
       ..
2064    1
385     1
3009    1
3997    1
2968    1
Name: drivingStyle, Length: 19009, dtype: object

In [ ]:
test_y[test_y=='AggressiveStyle']=0
test_y[test_y=='EvenPaceStyle']=1
test_y

1595    1
8415    1
2530    1
5554    1
3191    1
       ..
1623    1
4415    1
1404    1
208     1
3727    1
Name: drivingStyle, Length: 4753, dtype: object

Запуск Duet

In [2]:
import syft as sy
duet = sy.launch_duet()
sy.logging(file_path="./syft_do.log")

Exception: ignored

In [ ]:
import torch

# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df).float().to(device)

Загрузка тензоров в Duet store

In [ ]:
X_train = df_to_tensor(train_X)
torch.Tensor.tag(X_train, 'Train X')
torch.Tensor.send(X_train, duet, searchable=True)

Y_train = df_to_tensor(train_y.to_numpy(dtype=np.float64))
torch.Tensor.tag(Y_train, 'Train Y')
torch.Tensor.send(Y_train, duet, searchable=True)

X_test = df_to_tensor(test_X)
torch.Tensor.tag(X_test, 'Test X')
torch.Tensor.send(X_test, duet, searchable=True)

Y_test = df_to_tensor(test_y.to_numpy(dtype=np.float64))
torch.Tensor.tag(X_test, 'Test Y')
torch.Tensor.send(X_test, duet, searchable=True)

In [ ]:
train_num = sy.lib.python.Int(X_train.shape[0])
train_num.tag("train_num")
train_num.send(duet, searchable=True)

In [ ]:
duet.store.pandas

In [ ]:
duet.requests.pandas

In [ ]:
duet.requests[0].accept()

In [ ]:
duet.requests.add_handler(
    name="cuda_is_available",
    action="accept"
)

duet.requests.add_handler(
    name="loss",
    action="accept",
    timeout_secs=-1,  # no timeout
    print_local=True  # print the result in your notebook
)

duet.requests.add_handler(
    name="train_size",
    action="accept"
)

duet.requests.add_handler(
    name="inference",
    action="accept"
)

duet.requests.add_handler(
    name="model_download",
    action="accept"
)

In [ ]:
duet.requests.handlers